# Simple Blending

This is adapted from 

In [1]:
import sys
import socket
computer_name = socket.gethostname()
if computer_name == 'Alexs-MacBook-Pro.local':
    base_path = "/Users/alexsutherland/Documents/Programming/Python/Kaggle/Titanic---2015"
else:    
    base_path = 'C:\Users\Lundi\Documents\Programming\Python\Kaggle\Titanic - 2015'
sys.path.append(base_path)
sys.path.append(base_path + "\Stacked Generalization")

import TitanicPreprocessor as tp
import TitanicPredictor as tpred
import metaLearning as metaLearn
meta_learn = metaLearn.metaLearning()
import sklearn.ensemble as skl_ensemble
import sklearn.linear_model as skl_lm
import sklearn.grid_search as skl_gs
import sklearn.cross_validation as skl_cv
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

/Users/alexsutherland/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


Creating the n-folds

In [29]:
X, y, X_submission, X_test_ids = tp.getData()

n_folds = 10
skf = list(skl_cv.StratifiedKFold(y, n_folds))

Preparing the classifiers

In [13]:
clfs = [skl_ensemble.RandomForestClassifier(n_estimators=100, n_jobs=-1, criterion='gini'),
        skl_ensemble.RandomForestClassifier(n_estimators=100, n_jobs=-1, criterion='entropy'),
        skl_ensemble.ExtraTreesClassifier(n_estimators=100, n_jobs=-1, criterion='gini'),
        skl_ensemble.ExtraTreesClassifier(n_estimators=100, n_jobs=-1, criterion='entropy'),
        skl_ensemble.GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=50)]

Looping through all classifiers and all k-folds, the entire training dataset is predicted using 10-fold CV:

In [35]:
print "Creating train and test sets for blending."

dataset_blend_train = np.zeros((X.shape[0], len(clfs)))
dataset_blend_test = np.zeros((X_submission.shape[0], len(clfs)))

for j, clf in enumerate(clfs):
    print j, clf
    dataset_blend_test_j = np.zeros((X_submission.shape[0], len(skf)))
    for i, (train, test) in enumerate(skf):
        print "Fold", i
        X_train = X.ix[train,:]
        y_train = y.ix[train]
        X_test = X.ix[test,:]
        y_test = y.ix[test]
        clf.fit(X_train, y_train)
        y_submission = clf.predict_proba(X_test)[:,1]
        dataset_blend_train[test, j] = y_submission
        dataset_blend_test_j[:, i] = clf.predict_proba(X_submission)[:,1]
    dataset_blend_test[:,j] = dataset_blend_test_j.mean(1)

Creating train and test sets for blending.
0 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
1 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
2 ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_

The predictions of the training dataset are then blended using a logisitc regression model:

In [43]:
clf = skl_lm.LogisticRegression()
clf.fit(dataset_blend_train, y)
y_submission_proba = clf.predict_proba(dataset_blend_test)[:,1]
print "Linear stretch of predictions to [0,1]"
y_submission_proba = (y_submission_proba - y_submission_proba.min()) / (y_submission_proba.max() - y_submission_proba.min())
print "Generating classifications"
y_submission = map(lambda x: 1 if x > 0.5 else 0, y_submission_proba)

Linear stretch of predictions to [0,1]
Generating classifications


Outputting to file

In [47]:
test_output = pd.DataFrame([X_test_ids, y_submission]).T
test_output.columns = ['PassengerId','Survived']

test_output.to_csv('Data/test_output.csv', index=False)

Accuaracy was ~0.77 (not an improvement)